In [ ]:
# | default_exp tools.memory

In [1]:
# | export
from agentic.tools.base import BaseTool
from agentic.schemas import MemoryManagerParams
from typing import Dict, Any, List

class MemoryManagerTool(BaseTool):
    def __init__(self):
        self.memory = []  # Store conversation history

    def get_parameters_schema(self) -> Dict[str, Any]:
        """Return the OpenAI-compatible schema for memory_manager."""
        return {
                "type": "function",
                "function": {
                    "name": "memory_manager",
                    "description": "Manage conversation memory and token budget. Check usage, compress history, and optimize memory.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "action": {
                                "type": "string",
                                "enum": ["status", "compress", "optimize"],
                                "description": "Action: status (check usage), compress (force compression), optimize (auto-manage)"
                            },
                            "messages": {
                                "type": "array",
                                "description": "Conversation history for analysis"
                            }
                        },
                        "required": ["action"]
                    }
                }
            }

    def execute(self, **kwargs) -> Dict[str, Any]:
        """Manage conversation memory (status, compress, optimize)."""
        try:
            params = MemoryManagerParams(**kwargs)
            
            if params.action == "status":
                return {
                    "success": True,
                    "memory_size": len(self.memory),
                    "message": f"Memory contains {len(self.memory)} messages"
                }
            
            elif params.action == "compress":
                if not params.messages:
                    return {"error": "Messages required for compression"}
                
                # Simplified compression (keep last 10 messages)
                self.memory = params.messages[-10:]
                return {
                    "success": True,
                    "compressed_size": len(self.memory),
                    "message": "Memory compressed"
                }
            
            elif params.action == "optimize":
                if not params.messages:
                    return {"error": "Messages required for optimization"}
                
                # Simplified optimization (remove duplicates)
                seen = set()
                optimized = []
                for msg in params.messages:
                    msg_str = str(msg)
                    if msg_str not in seen:
                        seen.add(msg_str)
                        optimized.append(msg)
                
                self.memory = optimized
                return {
                    "success": True,
                    "optimized_size": len(self.memory),
                    "message": "Memory optimized"
                }
            
            return {"error": f"Unknown action: {params.action}"}
            
        except Exception as e:
            return {"error": f"Memory management failed: {str(e)}"}